In [ ]:
! python -m pip install mpi4py

In [ ]:
from mpi4py import MPI

In [ ]:
%%writefile week2.py

COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

###  Exercise 1

In [ ]:
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()
def f1():
    return print("Hello world")

def f2():
    return print("my rank is",RANK,"the total number of processors is :",SIZE)

def f3():
    if RANK==0:
        return print("my rank is : ",RANK,"my size is :",SIZE)

#4: we have to use MPI_FINALIZE to terminate the computation.

### Exercise 2

In [ ]:
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()
while True:
    if RANK == 0:
        x = int(input())
    else: 
        x = None

        x = COMM.bcast(x)
    if x< 0 :
        break
    print("rank is", rank,"data",data)

### Exercise 3

In [ ]:
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()
while (1):
    if RANK == 0:
        x = int(input())
        except ValueError:
            print("No integer entered")
            x = -1
        COMM.send(x,RANK+1)
    else : 
        x = COMM.recv(source = RANK-1)
        if RANK < SIZE - 1:
            if x < 0 : 
                x-=RANK 
                COMM.send(x +RANK, RANK+1)
    if x<0:
        break
    print("rank is", rank,"data",x)

###  Exercise 4

In [ ]:
COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

#1
n,m = int(input())
if rank == 0:
    A = np.random.rand(n, m)
else:
    A = None
A = COMM.bcast(A, root=0)
print("Rank of the matrix A is",RANK )


#2 
counts = [n // SIZE] * SIZE
remainder = n % SIZE
for i in range(remainder):
    counts[i] += 1
displs = [sum(counts[:i]) for i in range(SIZE)]
recv_buf = np.empty((counts[RANK], m), dtype=np.float64)
comm.Scatterv([A, counts, displs, MPI.DOUBLE], recv_buf, root=0)


#3,4,5
# calculate the number of rows to send to each processor
counts = [n // 2, n // 2, (n // 2) + (n % 2), (n // 2) + (n % 2)]
displs_1 = [0, m * (n // 2), m * ((n // 2) + (n % 2)), m * (n // 2) + (m // 2)]

blocklen = m // 2
stride = m
subarray_type = MPI.DOUBLE.Create_subarray((n, m), (counts[RANK], blocklen), (displs_1[RANK] // m, stride))
subarray_type.Commit()

if RANK == 0:
    COMM.Send([A, 1, subarray_type], dest=1)
    COMM.Send([A, 2, subarray_type], dest=2)
    COMM.Send([A, 3, subarray_type], dest=3)
elif RANK in (1, 2, 3):
    COMM.Recv(recv_buf, source=0)

# print the part of the matrix received by each processor
if RANK in (1, 2, 3):
    print("Rank",RANK," Matrix A received:" , str(recv_buf))

###  Exercise 5

In [ ]:
# in MatrixVectorMult.py file

###  Exercise 6

In [5]:
!pip install pyccel

     -------------------------------------- 383.4/383.4 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 67.8/67.8 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 55.0/55.0 kB 3.0 MB/s eta 0:00:00


In [7]:
%%file pi_.py
def f(a):
    return 4.0 / (1. + a * a)

def pi(n, h):
#     print("Execution of PI in parallel with", nb_tasks,"threads")

    for k in range(100):
        Pi_calc = 0.
        for i in range(n):
            x = h * (i + 0.5)
            Pi_calc += f(x)

        Pi_calc = h * Pi_calc
    return Pi_calc

Overwriting pi_.py


In [10]:
from pi_ import *
import time, math
if __name__ == '__main__':
    n = 300
    h = 1.0 / n
    from pyccel.epyccel import epyccel
    
    pyc_pi = epyccel(pi, language='c', accelerators=['openmp'])
    # start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time
    Pi_calc = pyc_pi.pi(n, h)
    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0
    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0
    # deviation between the estimated value and the calculated value of Pi
    Pi_estime = math.acos(-1)
    deviat = math.fabs(Pi_estime - Pi_calc)


ERROR at annotation (semantic) stage
pyccel:
 |error [semantic]: mod_zz3xb5nnsqzt_zz3xb5nnsqzt.py [1]| Function is missing a type annotation (pi)
 |fatal [semantic]: mod_zz3xb5nnsqzt_zz3xb5nnsqzt.py [6,17]| Undefined variable (n)



PyccelSemanticError: Semantic step failed

#### print the results

In [11]:
    print(
      "   Number of intervals        :", n,         "\n",
	  "   | Pi_estime - Pi_calcule | :", deviat,    "\n",
	  "   Temps elapsed              :", t_elapsed, "\n",
	  "   Temps CPU                  :", tcpu,      "\n",
    )

NameError: name 'deviat' is not defined